# サンプルエージェントの作成

行動判断を行う簡単なエージェントの作成方法を説明する. `readme.md`にあるように, シミュレータを実行するときに各エージェントがインポートされる際`getUserAgentClass`が呼ばれ, 実装したエージェントがシミュレータ内で返される. ここではPythonで自作したエージェントを実際にシミュレータ上で動かすまでを確認する. シミュレーションの流れが`基準シミュレータ 取扱説明書.pdf`の2.3項に実装すべきメソッドとともに記述されているので, よく確認しておくとよい. 大雑把に`(validate)->perceive->makeObs->deploy->control->behave->...`の順に実行されていく. 以下で, クラス定義からそれぞれのメソッドを説明しつつ実装していき, 最終的に初期行動判断モデルと対戦させて結果を確認する.

## クラスの定義

エージェントのインスタンスを作成するための抽象クラスを定義する. ここでは`MyAgent`という名前のクラスにする. エージェントの形として1機につき1つのエージェントを割り当てる分散方式と, 編隊全体で1つのエージェントを割り当てる中央集権方式がある. 分散方式の場合は`SingleAssetAgent`を, 中央集権方式の場合は`Agent`を継承する.

In [ ]:
# 分散方式の場合

from ASRCAISim1.libCore import SingleAssetAgent

class MyAgent(SingleAssetAgent):
    ...

In [ ]:
# 中央集権方式の場合

from ASRCAISim1.libCore import Agent

class MyAgent(Agent):
    ...

以下, 補助的な役割として, 慣性座標系と陣営座標系(進行方向が+x方向となるようにz軸まわりに回転させ、防衛ライン中央が原点となるように平行移動させた座標系)同士で変換しあうクラスと機体に対するコマンドを生成するための変数をまとめたクラスを示す. 好きに改変して使用するとよい.

In [ ]:
import numpy as np
from ASRCAISim1.libCore import Track3D


class TeamOrigin():
    #MotionStateを使用しても良いがクォータニオンを経由することで浮動小数点演算に起因する余分な誤差が生じるため、もし可能な限り対称性を求めるのであればこの例のように符号反転で済ませたほうが良い。
    #ただし、機体運動等も含めると全ての状態量に対して厳密に対称なシミュレーションとはならないため、ある程度の誤差は生じる。
    def __init__(self,isEastSider_,dLine):
        self.isEastSider=isEastSider_
        if(self.isEastSider):
            self.pos=np.array([0.,dLine,0.])
        else:
            self.pos=np.array([0.,-dLine,0.])
    def relBtoP(self,v):
        #陣営座標系⇛慣性座標系
        if(self.isEastSider):
            return np.array([v[1],-v[0],v[2]])
        else:
            return np.array([-v[1],v[0],v[2]])
    def relPtoB(self,v):
        #慣性座標系⇛陣営座標系
        if(self.isEastSider):
            return np.array([-v[1],v[0],v[2]])
        else:
            return np.array([v[1],-v[0],v[2]])

	
class ActionInfo():
     def __init__(self):
        self.dstDir=np.array([1.0,0.0,0.0]) #目標進行方向
        self.dstAlt=10000.0 #目標高度
        self.velRecovery=False #下限速度制限からの回復中かどうか
        self.asThrottle=False #加減速についてスロットルでコマンドを生成するかどうか
        self.keepVel=False #加減速について等速(dstAccel=0)としてコマンドを生成するかどうか
        self.dstThrottle=1.0 #目標スロットル
        self.dstV=300 #目標速度
        self.launchFlag=False #射撃するかどうか
        self.target=Track3D() #射撃対象
        self.lastShotTimes={} #各Trackに対する直前の射撃時刻

## `__init__`メソッドの実装

`modelConfig`と`instanceConfig`を渡す. 本シミュレータではモデルのパラメータセットを`modelConfig`という名称の`json`型変数で, インスタンス固有のパラメータセットを`instanceConfig`という名称の`json`型変数で取り扱うこととしている. なお, `modelConfig`とはFactoryに登録するためのエージェントに関する設定で, 

```json
{
    "Factory":{
        "Agent":{
            "modelName":{
                "class":"className",
                "config":{...}
            }
        }
    }
}
```

の"config"の部分に記載される{...}のdictが該当する. 例えば1[tick]ごとではない処理周期としたい場合などには, `基準シミュレータ 取扱説明書.pdf`の４.１.３項に従い, `modelConfig`に処理周期に関する記述を追加する.

```json
{
    "interval":{
        "unit":"time",
        "step":1.0,
        "perceive":0.1,
        "control":0.1,
        "behave":0.1
    },
    ...
}
```

上記の場合, "unit"を"time"で"step"を1.0にしているので, Agentの行動判断周期は1.0[s], "perceive", "control", "behave"の間隔は0.1[s]となる. こちらはjsonファイルとして保存しておき, シミュレーション実行時に`getUserAgentModelConfig`関数を呼んだときに読み込んで返す想定. なお, `modelConfig`を参照したい場合は`[]`でkeyを指定すれば`nljson`型で得られるので, Pythonプリミティブ型として扱う場合は`()`を付ける.

```Python
self.time_params = self.modelConfig['interval']()
```

また, `基準シミュレータ 取扱説明書.pdf`の3.4項にあるユーティリティを用いて確率的な選択やデフォルト値の設定も可能. その場合の出力はPythonプリミティブ型となるため`()`の付加は不要. 現バージョンでは乱数生成器には std::mt19937 しか使用できないが, `self.randomGen`として基底クラスで予め生成されているためこれを使用する.

```Python
self.withRandom = getValueFromJsonKR(self.modelConfig,"R",self.randomGen)
```

以下は中央集権方式の場合の実装. `modelConfig`から設定した"interval"を参照してattributeとして`time_params`と自軍が"Red"か"Blue"のどちらかを示す`own`と機体に対するコマンド([クラスの定義](#クラスの定義)で示した)をまとめた`actionInfos`を加えている. 実際にはその後の`makeObs`などにおいて参照したい情報をあらかじめ持っておくとよい.

In [ ]:
class MyAgent(Agent):
    def __init__(self,modelConfig,instanceConfig):
        super().__init__(modelConfig,instanceConfig) # 設定の読み込み
        if(self.isDummy):
            return # Factoryによるダミー生成のために空引数でのインスタンス化に対応させる
        self.time_params = self.modelConfig['interval']()
        self.own = self.getTeam()
        self.actionInfos=[ActionInfo() for _ in self.parents]

## `validate`メソッドの実装

modelConfigとして設定可能とするパラメータを選択し, 他の登場物に依存する初期化処理が必要な場合に使うメソッド. rulerやparentのobservablesに依存するものがあるような場合を想定している. 以下では戦域中心から場外ラインまでの距離, 戦域中心から防衛ラインまでの距離, 陣営座標系([クラスの定義](#クラスの定義)で示した)の変換クラスのインスタンスを取得している. このメソッドは必須ではない.

In [ ]:
def validate(self):
    #Rulerに関する情報の取得
    rulerObs=self.manager.getRuler()().observables()
    self.dOut=rulerObs["dOut"] # 戦域中心から場外ラインまでの距離
    self.dLine=rulerObs["dLine"] # 戦域中心から防衛ラインまでの距離
    self.teamOrigin=TeamOrigin(self.own==rulerObs["eastSider"],self.dLine) # 陣営座標系変換クラス定義

## `makeObs`メソッドの実装

observables(シミュレーション中に観測可能な情報)からObservation(行動判断を行うための情報に変換されたもの)を生成する関数. observablesとしては各種時刻情報, 味方機の状態(`基準シミュレータ 取扱説明書.pdf`の4.2項参照), 彼機のなかで味方の誰かが探知しているもののみ(`基準シミュレータ 取扱説明書.pdf`の4.2項参照), 味方誘導弾がある. 以下ではそれらの取得方法を示している.

味方機の情報としては以下が取得できる想定(`基準シミュレータ 取扱説明書.pdf`の4.2項参照).

| 名称 | 型 | 説明 |
| ---- | ---- | ---- |
|pos| array(double)| 位置ベクトル|
|vel| array(double)| 速度ベクトル|
|omega| array(double)| 角速度ベクトル|
|q |array(double)| 現在の姿勢.クォータニオンを実部⇒虚部の順に並べた4次元ベクトルとして記述.|
|qh| array(double)| 現在の局所水平座標系を表すクォータニオンを実部⇒虚部の順に並べた4次元ベクトルとして記述.|
|az| double| 現在の方位角(真北を0として東側を正)|
|el| double| 現在のピッチ角(下向きを正)|
|time| double| この MotionState を生成した時刻|

例えばPythonプリミティブな型として扱い取得したい場合は以下のように実装する. `modelConfig`の扱いと同様に`[]`でkeyを指定して`nljson`型で得た上で`()`を付ける.

```Python
self.ourMotions = []
for parent in self.parents.values():
    if(parent.isAlive()):
        self.ourMotions.append(parent.observables['motions']())
```

彼機の情報としては以下が取得できる想定(`基準シミュレータ 取扱説明書.pdf`の4.3項参照)

| 名称 | 型 | 説明 |
| ---- | ---- | ---- |
|truth| str| この3次元航跡が指す対象のAssetを特定するUUID(バージョン4)を表す文字列.|
|time| array(double) |この航跡を生成した時刻|
|pos |array(double) |位置ベクトル(慣性系)|
|vel |array(double) |速度ベクトル(慣性系)|
|buffer| array(object) |この3次元航跡と同一の対象を指すものとして外部から追加された3次元航跡のリスト.merge関数によって平均値をとる際に用いられる.|

例えばPythonプリミティブな型として扱いたい場合は以下のようにする(味方の誰かが探知しているもののみ).

```Python
for pIdx,parent in enumerate(self.parents.values()):
    if(parent.isAlive()):
        l=[Track3D(t) for t in parent.observables.at_p("/sensor/track")]
lastTrackInfo = [{'truth':track.truth.__str__(), 'time': track.time, 'pos': list(track.pos), 'vel':list(track.vel)} for track in l]
```

味方の誘導弾の情報としては以下が取得できる想定(`基準シミュレータ 取扱説明書.pdf`の4.7項や`基準モデル及びパラメータに関する資料.pdf`の1.5.4項参照)

| 名称 | 型 | 説明 |
| ---- | ---- | ---- |
|isAlive| bool |生存中か否か|
|hasLaunched| bool| 発射済みか否か|
|launchedT| double| 発射時刻. 未発射の場合は-1|
|mode| string| 目標の追尾状況であり,次の3種類のいずれか."guided": 母機から供給された航跡を使用, "self": 自身のセンサで捉えた航跡を使用, "memory":メモリトラックにより外挿中|
|target| object| 目標の3次元航跡.|
|motion| object| 現在の運動状態に関する値.|

例えばPythonプリミティブな型として扱いたい場合は以下のようにする(味方の誘導弾を射撃時刻の古い順にソート).

```Python
def launchedT(m):
    return m["launchedT"]() if m["isAlive"]() and m["hasLaunched"]() else np.inf
self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)
```

以下では特に何も加工しないが, ログとして残す情報をobservablesから選定して返している. ログとして残すにはjson形式で保存できなければいけないため, Pythonプリミティブな型としてjsonとして保存できる形で扱っている(`numpy.ndarray`の場合は自動的に`list`に変換するようになっているため, 保存可能.). また, メモリ制限があるので, 適宜調整されたい.

Observationを2次元画像や実数値ベクトルとして生成する方法の例が`/root/sample/modules/OriginalModelSample/OriginalModelSample/R5PyAgentSamle01M.py`にあるので, そちらも参照されたい. このメソッドは必須となる.

In [ ]:
import numpy as np
from ASRCAISim1.libCore import MotionState, Track3D

def makeObs(self):
    # 味方機(自機含む)
    self.ourMotion=[]
    self.ourObservables=[]

    for pIdx,parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            firstAlive=parent
            break
    for pIdx,parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            #残存していればobservablesそのもの
            self.ourMotion.append(parent.observables["motion"]())
            self.ourObservables.append(parent.observables)

        else:
            self.ourMotion.append({})
            #被撃墜or墜落済なら本体の更新は止まっているので残存している親が代理更新したものを取得(誘導弾情報のため)
            self.ourObservables.append(
                firstAlive.observables.at_p("/shared/fighter").at(parent.getFullName()))

    # 彼機(味方の誰かが探知しているもののみ)
    # 観測されている航跡を、自陣営の機体に近いものから順にソートしてlastTrackInfoに格納する。
    # lastTrackInfoは行動のdeployでも射撃対象の指定のために参照する。
    def distance(track):
        ret=-1.0
        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                myMotion=MotionState(parent.observables["motion"])
                tmp=np.linalg.norm(track.posI()-myMotion.pos)
                if(ret<0 or tmp<ret):
                    ret=tmp
        return ret
    for pIdx,parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            self.lastTrackInfo=sorted([Track3D(t) for t in parent.observables.at_p("/sensor/track")],key=distance) # type: ignore
            break

    # 味方誘導弾(射撃時刻が古いものから最大N発分)
    # 味方の誘導弾を射撃時刻の古い順にソート
    def launchedT(m):
        return m["launchedT"]() if m["isAlive"]() and m["hasLaunched"]() else np.inf
    self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)

    lastTrackInfo = [{'truth':track.truth.__str__(), 'time': track.time, 'pos': list(track.pos), 'vel':list(track.vel)} for track in self.lastTrackInfo]
    msls = [m() for m in self.msls]
    return {'ourMotion':self.ourMotion, 'lastTrackInfo': lastTrackInfo, 'msl': msls}

## `observation_space`メソッドの実装

状態空間を定義するメソッド. `makeObs`メソッドで返すObservationの形を定義する. 強化学習を実装する際は`makeObs`で返すObservationの型と一致させる必要がある. このメソッドは必須となる. 以下ではObservationの形式に従わず適当なダミーデータを返している. 

In [ ]:
from gymnasium import spaces

def observation_space(self):
    return spaces.Box(low=0.0,high=1.0,shape=(1,))

## `action_space`メソッドの実装

生成したObservationをもとに作成する行動の空間を定義するメソッド. `Agent/__init__.py`で定義した`policy`により行動を返す想定. 今回は制限時間以内に何らかの行動を返さなければここで定義された行動空間からランダムサンプリングされる. 以下では例として`spaces.Discrete`により0,1の整数値を定義するような実装としている. このメソッドは必須となる.

In [ ]:
from gymnasium import spaces

def action_space(self):
    return spaces.Discrete(2)

独自の実装としたい場合は`gymnasium.spaces.Space`を継承して`sample`メソッドをオーバーライドする.

In [ ]:
from gymnasium import spaces
from random import randint

class ActionSpace(spaces.Space):
    def sample(self):
        return randint(0,1)

`Agent/__init__.py`で観測されている航跡の中で自陣営の機体に最も近いものの速さを返す`policy`を定義する(あくまで一例で, Observationから後に続くdepolyメソッドに渡すactionとして適切なものを実装する想定).

In [ ]:
import numpy as np
from ASRCAISim1.policy import StandalonePolicy

class DummyPolicy(StandalonePolicy):
    def step(self,observation,reward,done,info,agentFullName,observation_space,action_space):
        vel = np.linalg.norm(observation['lastTrackInfo'][0]['vel'])
        return vel

## `deploy`メソッドの実装

なんらかの`policy`により`observation`から生成した`action`に対して`decision`や`commands`により行動を行うメソッド. このメソッドは1stepに1回実行される. `decision`はAgentクラス間に共通の行動意図を表現するためもの. `decision`の内容に関しては"Roll", "Throttle"等があるが, 詳細は`基準シミュレータ 取扱説明書.pdf`の4.5.3項を参照. 

- Roll

| 値 | 説明 |
| ---- | ---- |
|["Don’t care"]| ロール方向の回転について指定なしであることを表す.|
|["Angle",`value`]| ロール方向の回転について目標ロール角`value`への回転を意図していることを表す.|
|["Rate",`value`]| ロール方向の回転について指定角速度`value`での回転を意図していることを表す.|

- Horizontal

| 値 | 説明 |
| ---- | ---- |
|["Don’t care"]| 水平方向の旋回について指定なしであることを表す.|
|["Az_NED",`value`]| 水平方向の旋回について NED 座標系(慣性系)での指定方位`value`への旋回を意図していることを表す.|
|["Az_BODY",`value`]| 水平方向の旋回について機体座標系での指定方位`value`への旋回を意図していることを表す.|
|["Rate",`value`]|水平方向の旋回について指定角速度`value`での旋回を意図していることを表す.|

- Vertical

| 値 | 説明 |
| ---- | ---- |
|["Don’t care"]| 垂直方向の上昇・下降について指定なしであることを表す.|
|["El",`value`]| 垂直方向の上昇・下降について指定経路角(下向き正)`value`での上昇・下降を意図していることを表す.|
|["Pos",`value`]| 垂直方向の上昇・下降について目標高度`value`への上昇・下降を意図していることを表す.|
|["Rate",`value`]|垂直方向の上昇・下降について指定角速度`value`での上昇・下降を意図していることを表す.|

- Throttle

| 値 | 説明 |
| ---- | ---- |
|["Don’t care"]| 加減速について指定なしであることを表す.|
|["Vel",`value`]| 加減速について目標速度`value`への加減速を意図していることを表す.|
|["Accel",`value`]| 加減速について指定加速度`value`での加減速を意図していることを表す.|
|["Throttle",`value`]| 加減速について指定スロットルコマンド(0～1)`value`での加減速を意図していることを表す.|

- Fire

| 値 | 説明 |
| ---- | ---- |
| [`launchFlag`,`target`] | `launchFlag`は bool 型で、True のときに`target`に射撃する意図を持っていることを表す.`target`は Track3D の json表現とする.|

`decision`は特に使用しない場合は省略してもよい.


記述する`commands`の詳細は`基準シミュレータ 取扱説明書.pdf`の4.7.4項を参照.

| 名称 | 型 |説明|
| ---- | ---- |----|
|motion| object| 運動に関する値. 使用するFighterクラスにより異なる.|
|weapon| object| 射撃に関する値. `launch`: bool, 射撃するか否か. `target`:  object, 射撃する対象の3次元航跡のjson 表現|

observablesやObservationに応じてどのように動いてどのタイミングで射撃するかなどを決めるロジックを考案して実装する. [クラスの定義](#クラスの定義)で示した`ActionInfo`を活用するとよい(`self.actionInfos`).

例えば,

action, obsevables, Observationを加工, 必要な情報を取得->`actionInfos`をそれぞれの戦闘機で更新->`commands`を指定

という流れで実装するとよい.

以下では`action`は観測されている航跡の中で自陣営の機体に最も近いものの速さを表すとして, 彼機との距離に応じて接近したり遠ざかったりし, 地面に激突することを避けるために一定の高度になったら上昇, 一定の距離に達したら射撃するといったシンプルなロジックを実装している.

`基準モデルの簡易説明.pdf`の戦闘場面の定義において得点計算の仕方が記述してある(終了条件, 得点計算法など)ので, これらの情報も加味したうえでロジックを考案するとよい.

`root/sample/modules/OriginalModelSample/OriginalModelSample/R5PyAgentSample01M.py`なども参照されたい. このメソッドは必須.

In [ ]:
def deploy(self,action):
    for pIdx,parent in enumerate(self.parents.values()):
        if(not parent.isAlive()):
            continue

        self.observables[parent.getFullName()]["decision"]={
            "Roll":["Don’t care"],"Horizontal":["Az_BODY",0.0],
            "Vertical":["El",100.0],
            "Throttle":["Vel",150.0],
            "Fire":[False,Track3D().to_json()]
        }

    # observablesから必要な情報を取得
    tick = self.manager.getTickCount()
    time = self.manager.getTime()
    dist = {}
    for pIdx, parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            myMotion=MotionState(parent.observables["motion"])
            tracks = []
            for t in parent.observables.at_p("/sensor/track"):
                ret=-1.0
                track = Track3D(t)
                tmp=np.linalg.norm(track.posI()-myMotion.pos)
                if(ret<0 or tmp<ret):
                    ret = tmp
                tracks.append({'track':track, 'distance':ret})
            dist[pIdx] = tracks

    # actionInfosの更新
    for pIdx,parent in enumerate(self.parents.values()):
        if(not parent.isAlive()):
            continue
        # 自陣営の機体に最も近いものの速さを自分の速さに設定する
        self.actionInfos[pIdx].dstV = action
        
        # 自分に最も近い機体との距離が20000以下の場合は逃げるために距離をとり, それ以外の場合は接近する
        d = sorted(dist[pIdx], key=lambda x: x['distance'])[0]
        diff=MotionState(parent.observables["motion"]).pos-np.array(d['track'].posI())
        if d['distance'] <= 20000:
            print('escape')
            self.actionInfos[pIdx].dstDir=diff/np.linalg.norm(diff)
        else:
            print('move forward')
            self.actionInfos[pIdx].dstDir=-diff/np.linalg.norm(diff)

        # 目標進行方向のz成分が正で高度が5000以下の場合は上昇
        if self.actionInfos[pIdx].dstDir[2]>0 and np.array(MotionState(parent.observables["motion"]).pos)[2]>=-5000:
            print('up')
            self.actionInfos[pIdx].dstDir[2] = -self.actionInfos[pIdx].dstDir[2]

        # 自分に最も近い機体との距離が20000以上22000以下の場合は対象に対して射撃する
        self.actionInfos[pIdx].target = d['track']
        if d['distance'] >= 20000 and d['distance']<=22000:
            print('launch')
            self.actionInfos[pIdx].launchFlag=True


    # commandsに設定
    for pIdx,parent in enumerate(self.parents.values()):
        if(not parent.isAlive()):
            continue
        print(pIdx, MotionState(parent.observables["motion"]).pos, self.actionInfos[pIdx].dstV, self.actionInfos[pIdx].dstDir)
        self.commands[parent.getFullName()] = {
            "motion": { #機動の指定。以下の指定方法は一例。
                "dstDir": self.actionInfos[pIdx].dstDir, #進みたい方向を指定
                "dstV": self.actionInfos[pIdx].dstV #進みたい速度を指定
            },
            "weapon": {
                "launch": self.actionInfos[pIdx].launchFlag, #射撃要否を bool で指定
                "target": self.actionInfos[pIdx].target.to_json() #射撃目標の Track3D を json 化して指定
            }
        }

## `control`メソッドの実装

より細かく1tick単位の処理を行いたいときに実装するメソッド. `decision` and/or `commands`の複雑な生成処理を行う場合等に用いる. `commands`は`deploy`メソッドで計算してもよいが, ここでより高頻度に計算してもよい(どちらかで実装する必要はある). このメソッドは必須ではない.

## `behave`メソッドの実装

`control`メソッドの次に呼ばれる. `control`メソッドと同様により細かく1tick単位の処理を行いたいときに実装するメソッド. `decision` and/or `commands`の複雑な生成処理を行う場合等に用いる. このメソッドは必須ではない.

## `perceive`メソッドの実装

`behave`メソッドの次に呼ばれる. `control`メソッドと同様により細かく1tick単位の処理を行いたいときに実装するメソッド. `decision` and/or `commands`の複雑な生成処理を行う場合等に用いる. このメソッドは必須ではない.

## 全てをまとめる

今まで実装したメソッドをまとめて以下のように`MyAgent.py`として作成して, `./Agent`以下に保存する. なお, 中央集権方式をとっている.

In [ ]:
import numpy as np
from gymnasium import spaces
from ASRCAISim1.libCore import Agent, Fighter, MotionState, Track3D, Track2D


class MyAgent(Agent):

    class TeamOrigin():
        #陣営座標系(進行方向が+x方向となるようにz軸まわりに回転させ、防衛ライン中央が原点となるように平行移動させた座標系)を表すクラス。
        #MotionStateを使用しても良いがクォータニオンを経由することで浮動小数点演算に起因する余分な誤差が生じるため、もし可能な限り対称性を求めるのであればこの例のように符号反転で済ませたほうが良い。
        #ただし、機体運動等も含めると全ての状態量に対して厳密に対称なシミュレーションとはならないため、ある程度の誤差は生じる。
        def __init__(self,isEastSider_,dLine):
            self.isEastSider=isEastSider_
            if(self.isEastSider):
                self.pos=np.array([0.,dLine,0.])
            else:
                self.pos=np.array([0.,-dLine,0.])
        def relBtoP(self,v):
            #陣営座標系⇛慣性座標系
            if(self.isEastSider):
                return np.array([v[1],-v[0],v[2]])
            else:
                return np.array([-v[1],v[0],v[2]])
        def relPtoB(self,v):
            #慣性座標系⇛陣営座標系
            if(self.isEastSider):
                return np.array([-v[1],v[0],v[2]])
            else:
                return np.array([v[1],-v[0],v[2]])


    class ActionInfo():
        def __init__(self):
            self.dstDir=np.array([1.0,0.0,0.0]) #目標進行方向
            self.dstAlt=10000.0 #目標高度
            self.velRecovery=False #下限速度制限からの回復中かどうか
            self.asThrottle=False #加減速についてスロットルでコマンドを生成するかどうか
            self.keepVel=False #加減速について等速(dstAccel=0)としてコマンドを生成するかどうか
            self.dstThrottle=1.0 #目標スロットル
            self.dstV=300 #目標速度
            self.launchFlag=False #射撃するかどうか
            self.target=Track3D() #射撃対象
            self.lastShotTimes={} #各Trackに対する直前の射撃時刻


    def __init__(self,modelConfig,instanceConfig):
        super().__init__(modelConfig,instanceConfig) # 設定の読み込み
        if(self.isDummy):
            return # Factoryによるダミー生成のために空引数でのインスタンス化に対応させる
        self.time_params = self.modelConfig['interval']()
        self.own = self.getTeam()
        self.actionInfos=[self.ActionInfo() for _ in self.parents]


    def observation_space(self):
        return spaces.Box(low=0.0,high=1.0,shape=(1,))


    def action_space(self):
        return spaces.Box(low=150.0, high=450.0, shape=(1,))


    def validate(self):
        #Rulerに関する情報の取得
        rulerObs=self.manager.getRuler()().observables()
        self.dOut=rulerObs["dOut"] # 戦域中心から場外ラインまでの距離
        self.dLine=rulerObs["dLine"] # 戦域中心から防衛ラインまでの距離
        self.teamOrigin=self.TeamOrigin(self.own==rulerObs["eastSider"],self.dLine) # 陣営座標系変換クラス定義


    def makeObs(self):
        # 味方機(自機含む)
        self.ourMotion=[]
        self.ourObservables=[]

        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                firstAlive=parent
                break
        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                #残存していればobservablesそのもの
                self.ourMotion.append(parent.observables["motion"]())
                self.ourObservables.append(parent.observables)

            else:
                self.ourMotion.append({})
                #被撃墜or墜落済なら本体の更新は止まっているので残存している親が代理更新したものを取得(誘導弾情報のため)
                self.ourObservables.append(
                    firstAlive.observables.at_p("/shared/fighter").at(parent.getFullName()))

        # 彼機(味方の誰かが探知しているもののみ)
        # 観測されている航跡を、自陣営の機体に近いものから順にソートしてlastTrackInfoに格納する。
        # lastTrackInfoは行動のdeployでも射撃対象の指定のために参照する。
        def distance(track):
            ret=-1.0
            for pIdx,parent in enumerate(self.parents.values()):
                if(parent.isAlive()):
                    myMotion=MotionState(parent.observables["motion"])
                    tmp=np.linalg.norm(track.posI()-myMotion.pos)
                    if(ret<0 or tmp<ret):
                        ret=tmp
            return ret
        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                self.lastTrackInfo=sorted([Track3D(t) for t in parent.observables.at_p("/sensor/track")],key=distance) # type: ignore
                break

        # 味方誘導弾(射撃時刻が古いものから最大N発分)
        # 味方の誘導弾を射撃時刻の古い順にソート
        def launchedT(m):
            return m["launchedT"]() if m["isAlive"]() and m["hasLaunched"]() else np.inf
        self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)

        lastTrackInfo = [{'truth':track.truth.__str__(), 'time': track.time, 'pos': list(track.pos), 'vel':list(track.vel)} for track in self.lastTrackInfo]
        msls = [m() for m in self.msls]
        return {'ourMotion':self.ourMotion, 'lastTrackInfo': lastTrackInfo, 'msl': msls} #observables を加工し、所要の Observation を返す。


    def deploy(self,action):
        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue

            self.observables[parent.getFullName()]["decision"]={
                "Roll":["Don’t care"],"Horizontal":["Az_BODY",0.0],
                "Vertical":["El",100.0],
                "Throttle":["Vel",150.0],
                "Fire":[False,Track3D().to_json()]
            }

        # observablesから必要な情報を取得
        tick = self.manager.getTickCount()
        time = self.manager.getTime()
        dist = {}
        for pIdx, parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                myMotion=MotionState(parent.observables["motion"])
                tracks = []
                for t in parent.observables.at_p("/sensor/track"):
                    ret=-1.0
                    track = Track3D(t)
                    tmp=np.linalg.norm(track.posI()-myMotion.pos)
                    if(ret<0 or tmp<ret):
                        ret = tmp
                    tracks.append({'track':track, 'distance':ret})
                dist[pIdx] = tracks

        # actionInfosの更新
        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue
            # 自陣営の機体に最も近いものの速さを自分の速さに設定する
            self.actionInfos[pIdx].dstV = action
            
            # 自分に最も近い機体との距離が20000以下の場合は逃げるために距離をとり, それ以外の場合は接近する
            d = sorted(dist[pIdx], key=lambda x: x['distance'])[0]
            diff=MotionState(parent.observables["motion"]).pos-np.array(d['track'].posI())
            if d['distance'] <= 20000:
                print('escape')
                self.actionInfos[pIdx].dstDir=diff/np.linalg.norm(diff)
            else:
                print('move forward')
                self.actionInfos[pIdx].dstDir=-diff/np.linalg.norm(diff)

            # 目標進行方向のz成分が正で高度が5000以下の場合は上昇
            if self.actionInfos[pIdx].dstDir[2]>0 and np.array(MotionState(parent.observables["motion"]).pos)[2]>=-5000:
                print('up')
                self.actionInfos[pIdx].dstDir[2] = -self.actionInfos[pIdx].dstDir[2]

            # 自分に最も近い機体との距離が20000以上22000以下の場合は対象に対して射撃する
            self.actionInfos[pIdx].target = d['track']
            if d['distance'] >= 20000 and d['distance']<=22000:
                print('launch')
                self.actionInfos[pIdx].launchFlag=True


        # commandsに設定
        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue
            print(pIdx, MotionState(parent.observables["motion"]).pos, self.actionInfos[pIdx].dstV, self.actionInfos[pIdx].dstDir)
            self.commands[parent.getFullName()] = {
                "motion": { #機動の指定。以下の指定方法は一例。
                    "dstDir": self.actionInfos[pIdx].dstDir, #進みたい方向を指定
                    "dstV": self.actionInfos[pIdx].dstV #進みたい速度を指定
                },
                "weapon": {
                    "launch": self.actionInfos[pIdx].launchFlag, #射撃要否を bool で指定
                    "target": self.actionInfos[pIdx].target.to_json() #射撃目標の Track3D を json 化して指定
                }
            }


    def control(self):
        pass

## 投稿可能なプログラム一式としてまとめる

`Agent/__init__.py`を以下のように編集する. 中央集権方式なので, `isUserAgentSingleAsset`メソッドでは`False`を返すようにしている.

In [ ]:
import os,json
from ASRCAISim1.policy import StandalonePolicy

def getUserAgentClass(args={}):
    from .MyAgent import MyAgent
    return MyAgent


def getUserAgentModelConfig(args={}):
    configs=json.load(open(os.path.join(os.path.dirname(__file__),"config.json"),"r"))

    return configs


def isUserAgentSingleAsset(args={}):
    return False


class DummyPolicy(StandalonePolicy):
    def step(self,observation,reward,done,info,agentFullName,observation_space,action_space):
        vel = np.linalg.norm(observation['lastTrackInfo'][0]['vel'])
        return vel


def getUserPolicy(args={}):
    return DummyPolicy()

`config.json`を以下のように編集して`./Agent`以下に保存する(実装したアルゴリズムに応じて適切に作成する想定).

```json
{
    "interval":{
        "unit":"time",
        "step":1.0,
        "perceive":0.1,
        "control":0.1,
        "behave":0.1
    }
}
```

`__init__.py`の各関数に渡す引数として`args.json`を`./Agent`以下に保存する(実装したアルゴリズムに応じて適切に作成する想定).

```json
{
    "type": "original"
}
```

`./Agent`が以下のようなディレクトリ構造になっていることを確認.

```bash
Agent
├── __init__.py
├── args.json
├── config.json
└── MyAgent.py
```

## 対戦を実行する

作成したエージェントを初期行動判断モデルと戦わせる.

In [ ]:
import os

os.chdir('/workspace/') # /path/to/simulator_distに移動

In [ ]:
! python validate.py --agent-module-path ./Test --num-validation 1

適宜`--color`を"Blue"や"Red"に変えて陣営の種類に応じた行動が取れているかなどを確認する(実際の対戦では陣営の色はランダムに決まる). また, `--movie`と`--visualize`を`1`にして実際にどのように動いているかを確認するなりログを参考にするなりしてアルゴリズムをよりよくする.